In [ ]:
############################################################
############## (1) phrases extraction ######################
############################################################

######## install packages ########
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
!{sys.executable} -m pip install textacy
import spacy
import re
import textacy
from spacy.matcher import Matcher
import nltk
import pprint
from nltk import RegexpParser
from nltk import word_tokenize, pos_tag, ne_chunk, sent_tokenize
from nltk import Tree
import pandas as pd

In [ ]:
######## functions ############
nlp = spacy.load("en")
### no repetition version
def get_continuous_chunks(text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                current_chunk = [] 
        else:
            continue
            
    return continuous_chunk

def get_simple_chunks(text,chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    simple_chunk = []
    for subtree in chunked:
        if type(subtree) == Tree:
            simple_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
    return simple_chunk

def preprocess_s(sentence,myrule):
    token = word_tokenize(sentence)
    tags = pos_tag(token)
    cp = RegexpParser(myrule)
    result = cp.parse(tags)
    return result
def preprocess(document):
    sentences = sent_tokenize(document)
    token = [word_tokenize(sent) for sent in sentences]
    tags = [pos_tag(sent) for sent in token]
    grammar = "NP: {<DT>?<JJ>*<NN>}"
    cp = RegexpParser(grammar)
    result = [cp.parse(sent) for sent in tags]
    return result

In [ ]:
############## example sentence ############################
NP = r"""
  NP: {<DT|PP\$>?<JJ>*<NN.*>+} # noun phrase
  PP: {<IN><NP>}               # prepositional phrase
  VP: {<MD>?<VB.*><NP|PP>}     # verb phrase
  CLAUSE: {<NP><VP>}           # full clause
  ADJ: {<VB|VBD|VBN|VBZ><VBN|VBG>}
  NVB:{<NP><VB|VBD>}
  """
cp = RegexpParser(NP)
mysent = "Weather or environment or pollution caused coating defect in the low-voltage bushing showing blue phase - topping-up oil."
print(pos_tag(word_tokenize(mysent)))
print(get_continuous_chunks(mysent, cp.parse))

NP1 = r"""
  NP: {<DT|PP\$>?<JJ>*<NN.*>+} # noun phrase
  
  VP: {<MD>?<VB.*><NP|PP>}     # verb phrase
  CLAUSE: {<NP><VP>}           # full clause
  ADJ: {<VB|VBD|VBN|VBZ><VBN|VBG>}
  NVB:{<NP><VB|VBD>}
  """
cp1 = RegexpParser(NP1)
print(get_continuous_chunks(mysent, cp1.parse))
print(get_simple_chunks(mysent,cp1.parse))

In [ ]:
############### example 2######################
doc = "OIL LEAK - Bleed Valve to be replaced: Bleed Valve has been damaged in the past when removed by the use of stilsons, Bleed Valve to be replaced."
NP = "NP: {(<V\w+>|<NN\w?>)+.*<NN\w?>}"
chunks = preprocess_s(doc,NP)
print(chunks)
chunks[0].draw()
get_continuous_chunks(doc, cp.parse)
mysent = "Weather or environment or pollution caused coating defect in the low-voltage bushing showing blue phase - low-voltage bushing and high-voltage bushing requires painting."
myone = get_continuous_chunks("Weather or environment or pollution caused coating defect in the low-voltage bushing showing blue phase - low-voltage bushing and high-voltage bushing requires painting.", cp1.parse)
print(myone)
print(mytwo)
mytwo = causalmentionsent(mysent)
myha = mytwo[1][1].lower()
phrase = myone[4].lower()
print(phrase in myha) 
match = phrase in myha
if match:
    print(phrase)

here = pd.DataFrame({"cause":[],"effect":[]})
for i in range(len(mytwo)):
    aa = find_phrases(myone,mytwo[i][0])
    bb = find_phrases(myone,mytwo[i][1])
    thisisit = combos(aa,bb)
    if(all([len(b) == 1 for b in bb]) == True):
        thisisit = combos(aa,word_tokenize(bb))
    if(all([len(b) == 1 for b in aa]) == True):
        thisisit = combos(word_tokenize(aa),bb)
    if(all([len(b) == 1 for b in bb]) == True and all([len(b) == 1 for b in aa]) == True):
        thisisit = combos(word_tokenize(aa),word_tokenize(bb))
        
    for combo in thisisit:
      new_row = {"cause":combo[0],"effect":combo[1]}
      here = here.append(new_row, ignore_index=True)
print(here.head())
print(combos(find_phrases(myone,mytwo[1][0]),find_phrases(myone,mytwo[1][1])))
combos(find_phrases(myone,mytwo[i][0]),word_tokenize(find_phrases(myone,mytwo[1][1])))

In [ ]:
#######################################################
###########  (2) Causal Mentions   ####################
#######################################################
####
#### import packages
####
import os
import sys
import re
import time

In [ ]:
############# causal rules #####################
p_after = re.compile(r"After ([a-zA-Z\'\- ]+?),\s([\w\'\- ]+)\s?[\.,\?]")
p_because = re.compile(r"([\w\'\- ]+?)because(?!\sof) ([\w\'\- ]+)\s?[\.,\?]")
p_due_to_1 = re.compile(r"([\w\'\- ]+)[was|were|is|are]?due to\s([\w\'\- ]+)\s?[\.,\?]")
p_due_to_2 = re.compile(r"([\w\'\- ]+),\sdue to\s([\w\'\- ]+)\s?[\.,\?]") ##, needed? check later
p_because_of_1 = re.compile(r"Because of ([\w\'\- ]+?),\s([\w\'\- ]+)\s?[\.,\?]")
p_because_of_2 = re.compile(r"([\w\'\- ]+)because of\s([\w\'\- ]+)\s?[\.,\?]")
p_cause = re.compile(r"([\w\'\- ]+)\scause(?!\sby)\s([\w\'\- ]+)\s?[\.,\?]")
p_caused = re.compile(r"([\w\'\- ]+)\scaused(?!\sby)\s([\w\'\- ]+)\s?[\.,\?]")
p_causes = re.compile(r"([\w\'\- ]+)\scauses(?!\sby)\s([\w\'\- ]+)\s?[\.,\?]")
p_caused_by = re.compile(r"([\w\'\- ]+)[was|were|is|are]?caused by\s([\w\'\- ]+)\s?[\.,\?]")
p_lead_to = re.compile(r"([\w\'\- ]+)lead to ([\w\'\- ]+)\s?[\.,\?]")
## dash,colon
p_require = re.compile(r"([\w\'\- ]+)\srequire[s|d]?([\w\'\- ]+)\s?[\.,\?]")
p_so = re.compile(r"([\S ]+?)so(?!\sof) ([\S ]+)\s?[\.,\?]")
p_result_in = re.compile(r"([\w\'\- ]+)\sresult\sin\s([\w\'\- ]+)\s?[\.,\?]")
p_results_in = re.compile(r"([\w\'\- ]+)\sresults\sin\s([\w\'\- ]+)\s?[\.,\?]")
p_resulted_in = re.compile(r"([\w\'\- ]+)\sresulted\sin\s([\w\'\- ]+)\s?[\.,\?]")
p_as_a_consequence=re.compile(r"([\S ]+?)as a consequence(?!\sof) ([\S ]+)\s?[\.,\?]")
p_as_a_result=re.compile(r"([\S ]+?)as a result(?!\sof) ([\S ]+)\s?[\.,\?]")
#p_for_reason = re.compile(r"([\S ]+?)for reason(?!\sthat) ([\S ]+)\s?[\.,\?]")
p_dash = re.compile(r"([\w\'\- ]+)\s\-\s([\w\'\- ]+)\s?[\.,\?]")
p_colon = re.compile(r"([\w\'\- ]+)\:\s([\w\'\- ]+)\s?[\.,\?]")

triggers = ["After"," because of"," because"," is caused by"," was caused by"," are caused by"," were caused by",
           " caused"," causes"," cause"," is due to"," are due to"," was due to"," were due to"," result in"," resulted in",
           " results in"," as a consequence of"," as a consequence"," as a result of"," as a result"," so that"," so",
           " required"," requires"," require"]
# Some notes: [] indicate a character class, which matches only a single character, e.g. gr[ae]y does not match graay
#             () for grouping
#             ^ negated character classes, if not right after the opening bracket, its just a character
#             ? optional items, e.g. Nov(ember)? matches Nov and November, colou?r matches colour and color
#            ?,*,+ repeating character class, ([0-9])\1+ matches 222, repeated same number
#             + 1+1=2 matches to 111=2 repeat the preceding token once or more
#             * repeat the preceding token >=0 times
#             (?!)negative lookahead, the regex only match when the capturing group not match
#             \d shorthand for [0-9]
#             \w shorthand for word character [A-Za-z0-9_]
#             \s whitespace character includes [ \t\r\n\f] a space, a tab, a carriage return, a line feed, or a form feed
#             \D equivalently [^\d]
#             \W equivalently [^\w]
#             \S equivalently [^\s]
#             [^\d\s] matches any character neither a digit nor whitespace
#             [\D\S] mathces any character that is either not a digit or is not whitespace



In [ ]:
################## functions to pick up the causal mentions from causal rules
def test():
    
   # line = r"after wesd Afghan vote, comp-laints because lead-to that' of fraud, surfacev ? jadsklfj?"
    line = r"low oil caused outage."
   # match = p_because.search(line)
    match = p_caused.search(line)
    if match:
        print("Match Rex ...")
        print(match.group(0))
        print(match.group(1))
        print(match.group(2))
    else:
        print("Cannot Match ...")

In [ ]:
## for a single sentence

## for a document

## segment sentences : doctype = 1 represent csv, 0 represent txt
##                     document is the dataframe, converted from csv
def sentencesplittxt(doctype,document,sentencefile):
    if doctype == 0:
        input = open(document, 'r')
        all_the_text = input.read()
        input.close()
    
        text = all_the_text.rstrip() ## remove whitespace at the end of the string from all texts
        text = all_the_text.lstrip() ## remove whitespace at the beginning of the string 
        text = " ".join(text.split()) ## split a string into a list s.t. each word is a list item, then join them into a string using a space as a separator
    
    #Math the end of sentences
        end = re.compile(r"[.!?.]+")
        sentences = re.split(end, text)[:-1]
        output = open(sentencefile, 'w')
        for sentence in sentences:
            output.writelines(sentence+'.'+'\n')

        output.close()
        

## causal mention extraction for a single document (a list) with multiple sentences 
##                          length = len(segment), number of sentences
def causalmentiondoc(document,length):
    listofpair=[]
    i = 0
    
    while i < length:
        causeeffect=[]
        # (1) after
        match = p_after.search(str(document[i])) 
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (2) because    
        match = p_because.search(str(document[i])) 
        if match:
            causeeffect.append((match.group(2),match.group(1)))
        # (3) due to 1
        match = p_due_to_1.search(str(document[i])) 
        if match:
            causeeffect.append((match.group(2),match.group(1)))
        # (4) due to 2
        match = p_due_to_2.search(str(document[i])) 
        if match:
            causeeffect.append((match.group(2),match.group(1)))
        # (5) because of 1
        match = p_because_of_1.search(str(document[i])) 
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (6) because of 2
        match = p_because_of_2.search(str(document[i]))
        if match:
            causeeffect.append((match.group(2),match.group(1)))
        # (7) cause
        match = p_cause.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        match = p_caused.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        match = p_causes.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (8) caused by
        match = p_caused_by.search(str(document[i]))
        if match:
            causeeffect.append((match.group(2),match.group(1)))
        # (9) lead to
        match = p_lead_to.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (10) require
        match = p_require.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (11) so
        match = p_so.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (12) result in
        match = p_result_in.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (13) results in
        match = p_results_in.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (14) resulted in
        match = p_resulted_in.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (15) as a consequence
        match = p_as_a_consequence.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (16) as a result
        match = p_as_a_result.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (17) dash
        match = p_dash.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
        # (18) colon
        match = p_colon.search(str(document[i]))
        if match:
            causeeffect.append((match.group(1),match.group(2)))
            
        listofpair.append(causeeffect)
        i += 1    
        
    #listofpair = list(filter(None,listofpair))
    return listofpair  


### if only for sentences:
def causalmentionsent(sent):
   
    causeeffect=[]
        # (1) after
    match = p_after.search(str(sent)) 
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (2) because    
    match = p_because.search(str(sent)) 
    if match:
         causeeffect.append((match.group(2),match.group(1)))
        # (3) due to 1
    match = p_due_to_1.search(str(sent)) 
    if match:
        causeeffect.append((match.group(2),match.group(1)))
        # (4) due to 2
    match = p_due_to_2.search(str(sent)) 
    if match:
        causeeffect.append((match.group(2),match.group(1)))
        # (5) because of 1
    match = p_because_of_1.search(str(sent)) 
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (6) because of 2
    match = p_because_of_2.search(str(sent))
    if match:
        causeeffect.append((match.group(2),match.group(1)))
        # (7) cause
    match = p_cause.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
    match = p_caused.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
    match = p_causes.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (8) caused by
    match = p_caused_by.search(str(sent))
    if match:
        causeeffect.append((match.group(2),match.group(1)))
        # (9) lead to
    match = p_lead_to.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (10) require
    match = p_require.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (11) so
    match = p_so.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (12) result in
    match = p_result_in.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (13) results in
    match = p_results_in.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (14) resulted in
    match = p_resulted_in.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (15) as a consequence
    match = p_as_a_consequence.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (16) as a result
    match = p_as_a_result.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (17) dash
    match = p_dash.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))
        # (18) colon
    match = p_colon.search(str(sent))
    if match:
        causeeffect.append((match.group(1),match.group(2)))  
        
   
    return causeeffect     
        
def causalmentions(doctype,document,sentencefile):
    end = re.compile(r"[.!?.]+")
    output = open(sentencefile, 'w')
    if doctype == 1:
        listofdocs = df["logs"].tolist()
        seglength = []
        mentions = []
        for entry in listofdocs:
            text = entry.rstrip()
            text = " ".join(text.split())
            sentences = re.split(end, text)[:-1]
            seglength.append(len(sentences))
            segment = []
            
            for sentence in sentences:
                segment.append(sentence+'.')
            
            mentions.append(causalmentiondoc(segment,len(sentences)))
    
    return mentions
                
            

        
        
def phrase_in(phrase,mention):
    phrase = phrase.lower()
    mention = mention.lower()
    return phrase in mention
def find_phrases(phrases,mention):
    gamma = []
    mention = mention.lower()
    for phrase in phrases:
        phrase = phrase.lower()
      #  print(phrase in mention) 
        match = phrase in mention
        if match:
            gamma.append(phrase)
            #gamma +=(phrase,)
    if len(gamma) == 0:
        if(word_tokenize(mention) == mention):
            gamma.append(word_tokenize(mention))
        else:
            gamma = mention
    return gamma   

def combos(lista,listb):
    combos = []
    for i in lista:
        for j in listb:
            combos.append([i,j])
    return combos
        
#def causalphrases(phrases,causalmentions):
    
## for a file of documents (preprocess to pick sentences/docs with trigger words)
#def causalmentions_file(sentencefile,causalityfile):
    
    ## define terms 
#    input = open(sentencefile, 'r')
#    sentences = input.readlines()
#    input.close()
#    os.remove(sentencefile)
#    output = open(causalityfile,'w')

In [ ]:
######## eample 3 ###########
hard = "The oil leak caused the machine to stop working. Need to change the tank and add 2 litres oil."
hardtext = hard.strip()
hardtext = " ".join(hardtext.split())

end = re.compile(r"[.!?.]+")
hardtext = re.split(end, hardtext)[:-1]
hardtext
#for sent in hardtext:
#    print(sent+'.'+'\n')

In [ ]:
############ If the data is csv rather than txt
import pandas as pd
df = pd.read_csv(r'./Documents/PhD paper 2/data/texts2.csv')
listofdocs = df["logs"].tolist()
tryresult = []
phrasechunks = []
step3 = []
df = pd.DataFrame(columns=["cause","effect","doc_ind","sentence_ind"])
document_index = 0
for entry in listofdocs:
    document_index += 1
    text = entry.rstrip()
    text = " ".join(text.split())
    sentences = re.split(end, text)[:-1]
    segment = []
    phrases = []
    sentc = []
    sentence_index = 0
    for sentence in sentences:
        sentence_index += 1
        segment.append(sentence+'.')
        ## betaresult: causal mentions in a single sentence
        betaresult = causalmentionsent(sentence+'.')
       # mention = []
       # multimention = []
        ## remove trigger words 
        
        for word in triggers:
            sentence = sentence.replace(word,",")
        
        ## alpharesult: phrases parsed by rules in a single sentence with triggers removed
        alpharesult = get_continuous_chunks(sentence+'.', cp.parse)
        ## phrases: the phrases parsed for a document
        phrases.append(get_continuous_chunks(sentence+'.', cp.parse))
      
        if len(betaresult) > 0:
            i = 0
            while i < len(betaresult): ## when there are more than one pair of causal mentions extracted from one sentence
                ## for ith pair of causal mentions in a single sentence
                ## beta is one of the mention in the ith pair for a single sentence
                #for beta in betaresult[i]: 
                    ## parsedmention is the mention that is further refined by the causal phrases alpharesult
                #    parsedmention = find_phrases(alpharesult,beta)
                    ## mention
                 #   mention.append(parsedmention)
            
                tempcause = find_phrases(alpharesult,betaresult[i][0])
                tempeffect = find_phrases(alpharesult,betaresult[i][1])
                sentencecombo = combos(tempcause,tempeffect)
                
                for combo in sentencecombo:
                    new_row = {"cause":combo[0],"effect":combo[1],"doc_ind":document_index,"sentence_ind":sentence_index}
                    df = df.append(new_row, ignore_index=True)
                
                i += 1
         

In [ ]:
df.head()

In [ ]:
print(df)
df.to_csv(r'./Documents/PhD paper 2/data/cmoftexts2.csv')

In [ ]:
import xml.dom.minidom as readxml
def xmltodf(path):
    myfile = readxml.parse(path)
    ## we extract the events and their indices
    events = myfile.getElementsByTagName("event")
    event = []
    eventid = []
    for e in events:
        event.append(e.attributes["string"].value)
        eventid.append(e.attributes["eiid"].value)
    ## now extract the times and their indices
    print(len(event))
    times = myfile.getElementsByTagName("timex")
    time = []
    timeid = []
    for t in times:
        time.append(t.attributes["text"].value)
        timeid.append(t.attributes["tid"].value)
    ## now extract event pairs
    myitem = myfile.getElementsByTagName("tlink")
    event1 = []
    event2 = []
    relation = []
    for item in myitem:
        event1.append(item.attributes["event1"].value)
        event2.append(item.attributes["event2"].value)
        relation.append(item.attributes["relation"].value)
    ## now remove those pairs with times
    i = 0
    subset = []
    while i < len(event1):
        exist1 = event1[i] in timeid
        exist2 = event2[i] in timeid
        if exist1 or exist2:
            subset.append(False)
        else:
            subset.append(True)
        i += 1
    event1 = [x for x,y in zip(event1,subset) if y]
    event2 = [x for x,y in zip(event2,subset) if y]
    relation = [x for x,y in zip(relation,subset) if y]
    ## now replace the indices by the texts
    i = 0
    verb1 = []
    verb2 = []
    while i < len(event1):
        j = 0
        for j in range(len(event)):
            if event1[i] == eventid[j]:
                verb1.append(event[j])
            if event2[i] == eventid[j]:
                verb2.append(event[j])
            j += 1
        i += 1
    mydata = {"event1":verb1,"event2":verb2,"relation":relation}
    df = pd.DataFrame(data = mydata)
    return(df)


## create a cause-effect dataframe from caevodataframe(with document index):
def caevodf(caevoresult):
    ## vague relation not meaningful
    hdf = caevoresult[caevoresult["relation"] != "VAGUE"]
    i = 0
    cause = []
    effect = []
    doc_ind = []
    while i < len(hdf["relation"]):
        j = list(hdf.index)[i]
        if hdf["relation"][j] == "BEFORE":
            cause.append(hdf["event1"][j])
            effect.append(hdf["event2"][j])
           # doc_ind.append(hdf["doc_ind"][j])
            doc_ind.append(0)
        elif hdf["relation"][j] == "AFTER":
            cause.append(hdf["event2"][j])
            effect.append(hdf["event1"][j])
           # doc_ind.append(hdf["doc_ind"][j])
            doc_ind.append(0)
        else: ## simultaneous or included
            cause.append(hdf["event1"][j])
            effect.append(hdf["event2"][j])
           # doc_ind.append(hdf["doc_ind"][j])
            doc_ind.append(0)
            cause.append(hdf["event2"][j])
            effect.append(hdf["event1"][j])
           # doc_ind.append(hdf["doc_ind"][j])
            doc_ind.append(0)
        i += 1
    caevodata = {"cause":cause,"effect":effect,"doc_ind":doc_ind}
    mydf = pd.DataFrame(data = caevodata)
    return mydf

In [ ]:
xmldf = xmltodf("./caevo-master/src/test/resources/conservator.txt.info.xml")
xmldf["doc_ind"] = 0
xmldf.head()

In [ ]:
cdf = caevodf(xmldf)
print(cdf)

In [ ]:
filterdf = pd.DataFrame(data={"cause":df["cause"],"effect":df["effect"],"doc_ind":df["doc_ind"],"sentence_ind":df["sentence_ind"]})
ind = 0
for text in df["cause"]:
    filterdf["cause"][ind] = rm_stop(text)
    ind += 1
ind = 0
for text in df["effect"]:
    filterdf["effect"][ind] = rm_stop(text)
    ind += 1
df.head()
filterdf.head()

In [ ]:
tagdf = pd.DataFrame(data={"cause":df["cause"],"effect":df["effect"],"doc_ind":df["doc_ind"],"sentence_ind":df["sentence_ind"]})
tagdf = wntagset(tagdf,df,filterdf)
tagdf.head()

In [ ]:
#### orderevent function when ignoring document index

def orderedevent2(caevodf,filterdf):
    ## lemmatize the caevodf:
    i = 0
    acause = []
    aeffect = []
    for i in list(caevodf.index):
        thisc = lemmatizer.lemmatize(caevodf["cause"][i],"v")
        thise = lemmatizer.lemmatize(caevodf["effect"][i],"v")
        yes1 = "cause" in thisc or "cause" in thise
        yes2 = "require" in thisc or "require" in thise
        agree = yes1 or yes2
        if agree == False:
            acause.append(thisc)
            aeffect.append(thise)
    creatdf = {"cause":acause,"effect":aeffect}
    lemmacaevodf = pd.DataFrame(data = creatdf)
    addcause = []
    addeffect = []
    adddoc = []
    addsent = []
    
    
    for k in list(lemmacaevodf.index):
        add = False
        for h in list(filterdf.index):
            exist1 = lemmacaevodf["cause"][k] in filterdf["cause"][h]
            exist2 = lemmacaevodf["cause"][k] in filterdf["effect"][h]
            exist3 = lemmacaevodf["effect"][k] in filterdf["cause"][h]
            exist4 = lemmacaevodf["effect"][k] in filterdf["effect"][h]
            cond1 = exist1 and exist3 ## cause-effect (caevo) exist in a cause
            cond2 = exist1 and exist4 ## cause-effect exist
            cond3 = exist2 and exist3 ## cause-effect exist as effect-cause
            cond4 = exist2 and exist4 ## cause-effect exist in an effect
            existed = cond1 or cond2 or cond3 or cond4
            add = add or existed
        if add == False: ## if any of them is true, then the pair already exist
                    ## we should not change the existed pairs, otherwise we add this pair
            addcause.append(lemmacaevodf["cause"][k])
            addeffect.append(lemmacaevodf["effect"][k])
            adddoc.append("NA")
            addsent.append("NA")
            
    adddata = {"cause":addcause,"effect":addeffect,"doc_ind":adddoc,"sentence_ind":addsent}
    adddf = pd.DataFrame(data = adddata)

    outdf = filterdf.append(adddf)
    return outdf

In [ ]:
lemmacdf = orderedevent2(cdf,filterdf)
lemmacdf.head()

In [ ]:
print(lemmacdf.shape)
print(df.shape)
lemmacdf[lemmacdf.doc_ind == "NA"]

In [ ]:
lemmacdf.head()

In [ ]:
## add these new caevo pairs to tagdf
nowcause = []
noweffect = []
nowdoc = []
nowsent =[]
newpart = lemmacdf[lemmacdf.doc_ind == "NA"]
i=0
while i < 1129:
    if i < 1117:
        nowcause.append(tagdf["cause"][i])
        noweffect.append(tagdf["effect"][i])
        nowdoc.append(tagdf["doc_ind"][i])
        nowsent.append(tagdf["sentence_ind"][i])
    else:
        nowcause.append(tagtuple_convert(pos_tag(word_tokenize(newpart["cause"][i-1117]))))
        noweffect.append(tagtuple_convert(pos_tag(word_tokenize(newpart["effect"][i-1117]))))
        nowdoc.append("NA")
        nowsent.append("NA")
    i += 1
nowdata = {"cause":nowcause,"effect":noweffect,"doc_ind":nowdoc,"sentence_ind":nowsent}
newtagdf = pd.DataFrame(data=nowdata)

In [ ]:
## adjust index of lemmacdf
newlemmacdf = pd.DataFrame(data = {"cause":list(lemmacdf["cause"]),
                                  "effect":list(lemmacdf["effect"]),
                                  "doc_ind":list(lemmacdf["doc_ind"]),
                                  "sentence_ind":list(lemmacdf["sentence_ind"])},
                          index = range(1129))
nouns = pairnouns(picknouns(newlemmacdf,newtagdf))
print(nouns)
hypers = hypernyms(nouns)
hypers.to_csv(r'./Documents/PhD paper 2/data/hyperstext2.csv')
print(hypers)
verbs = pickverbs(newlemmacdf,newtagdf)
classes = verbclass(verbs)
classes.to_csv(r'./Documents/PhD paper 2/data/classestext2.csv')
newlemmacdf.to_csv(r'./Documents/PhD paper 2/data/cmcaevotext2.csv')

In [ ]:
print(filterdf["cause"][1])
print(df["cause"][1])
print(subset_sentence(filterdf["cause"][1],df["cause"][1]))
print([pos_tag(word_tokenize(df["cause"][1]))[i] for i in subset_sentence(filterdf["cause"][1],df["cause"][1])])
haha = [pos_tag(word_tokenize(df["cause"][1]))[i] for i in subset_sentence(filterdf["cause"][1],df["cause"][1])]
print(tagtuple_convert(haha)) 
print(lemmacdf["cause"][1])
print(lemmacdf["effect"][1])
print(cdf)
print(tagtuple_convert(pos_tag(word_tokenize(cdf["cause"][0]))))
print(tagtuple_convert(pos_tag(word_tokenize(cdf["cause"][1]))))
print(tagtuple_convert(pos_tag(word_tokenize(cdf["effect"][0]))))
print(tagtuple_convert(pos_tag(word_tokenize(cdf["effect"][1]))))
ntcause = []
ntcause.append(tagtuple_convert(pos_tag(word_tokenize(cdf["cause"][1]))))
ntcause.append(tagtuple_convert(pos_tag(word_tokenize(cdf["cause"][0]))))
nteffect = []
nteffect.append(tagtuple_convert(pos_tag(word_tokenize(cdf["effect"][1]))))
nteffect.append(tagtuple_convert(pos_tag(word_tokenize(cdf["effect"][0]))))
ntdata = {"cause":ntcause,"effect":nteffect,"doc_ind":[5,7],"sentence_ind":["NA","NA"]}
nt = pd.DataFrame(data = ntdata,index = [143,144])
#print(nt)
newtagdf = tagdf
newtagdf = newtagdf.append(nt)
print(newtagdf)


In [ ]:
newlemmacdf = pd.DataFrame(data = {"cause":list(lemmacdf["cause"]),
                                  "effect":list(lemmacdf["effect"]),
                                  "doc_ind":list(lemmacdf["doc_ind"]),
                                  "sentence_ind":list(lemmacdf["sentence_ind"])},
                          index = range(145))
nouns = pairnouns(picknouns(newlemmacdf,newtagdf))
print(nouns)
hypers = hypernyms(nouns)
hypers.to_csv(r'./Documents/PhD paper 2/data/hypersbush.csv')
print(hypers)
verbs = pickverbs(newlemmacdf,newtagdf)
classes = verbclass(verbs)
classes.to_csv(r'./Documents/PhD paper 2/data/classesbush.csv')
newlemmacdf.to_csv(r'./Documents/PhD paper 2/data/filterbush.csv')

In [ ]:
#### combine results from caevo to extracted causal events
## caevodf has a column of document index(use xmftodf to each document and concatenate them)
## docindex = [1:...]
## this function not tested yet

def orderedevent(caevodf,filterdf,lemmacaevodf):
    ## lemmatize the caevodf:
    i = 0
    for i in list(caevodf.index):
        lemmacaevodf["cause"][i] = lemmatizer.lemmatize(caevodf["cause"][i],"v")
        lemmacaevodf["effect"][i] = lemmatizer.lemmatize(caevodf["effect"][i],"v")
    addcause = []
    addeffect = []
    adddoc = []
    addsent = []
    
    docindex = np.unique(filterdf["doc_ind"])
    ## make a string of filterdf cause, effect with only verbs
    for j in docindex:
        tempdf = filterdf[filterdf["doc_ind"] == j]
        temppair = lemmacaevodf[lemmacaevodf["doc_ind"] == j]
       # k = 0
        
        if len(temppair["doc_ind"]) > 0:
            for k in list(temppair.index):
                add = False
                for h in list(tempdf.index):
                    exist1 = temppair["cause"][k] in tempdf["cause"][h]
                    exist2 = temppair["cause"][k] in tempdf["effect"][h]
                    exist3 = temppair["effect"][k] in tempdf["cause"][h]
                    exist4 = temppair["effect"][k] in tempdf["effect"][h]
                    cond1 = exist1 and exist3
                    cond2 = exist1 and exist4
                    cond3 = exist2 and exist3
                    cond4 = exist2 and exist4
                    existed = cond1 or cond2 or cond3 or cond4
                    add = add or existed
                if add == False: ## if any of them is true, then the pair already exist
                    ## we should not change the existed pairs, otherwise we add this pair
                    addcause.append(temppair["cause"][k])
                    addeffect.append(temppair["effect"][k])
                    adddoc.append(j)
                    addsent.append("NA")
    adddata = {"cause":addcause,"effect":addeffect,"doc_ind":adddoc,"sentence_ind":addsent}
    adddf = pd.DataFrame(data = adddata)

    outdf = filterdf.append(adddf)
    return outdf
                    
      

In [ ]:
filterdf = pd.DataFrame(data={"cause":df["cause"],"effect":df["effect"],"doc_ind":df["doc_ind"],"sentence_ind":df["sentence_ind"]})
ind = 0
for text in df["cause"]:
    filterdf["cause"][ind] = rm_stop(text)
    ind += 1
ind = 0
for text in df["effect"]:
    filterdf["effect"][ind] = rm_stop(text)
    ind += 1
print(filterdf)
print(df)
#print([pos_tag(word_tokenize(df["effect"][0]))[i] for i in subset_sentence(filterdf["cause"][0],df["cause"][0])])
tagdf = pd.DataFrame(data={"cause":df["cause"],"effect":df["effect"],"doc_ind":df["doc_ind"],"sentence_ind":df["sentence_ind"]})
tagdf = wntagset(tagdf,df,filterdf)
print(tagdf)
nouns = pairnouns(picknouns(filterdf,tagdf))
print(nouns)
hypers = hypernyms(nouns)
print(hypers)
verbs = pickverbs(filterdf,tagdf)
classes = verbclass(verbs)
print(classes)
